In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp
import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

In [2]:
SEED = 42

In [3]:
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
tfd = tfp.distributions

In [4]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
df_global = pd.read_csv('splitted/Total.csv')
df_global.drop(columns=[col for col in df_global.columns if col.startswith('time')], inplace=True, )

In [6]:
df_global.head()

,y_generation biomass,y_generation nuclear,y_generation other,y_generation other renewable,y_generation solar,y_generation waste,y_price actual,y_generation fossil,y_generation hydro,y_generation wind,...,x_temp_min_Bilbao,x_temp_max_Bilbao,x_pressure_Bilbao,x_humidity_Bilbao,x_wind_speed_Bilbao,x_wind_deg_Bilbao,x_rain_1h_Bilbao,x_rain_3h_Bilbao,x_snow_3h_Bilbao,x_clouds_all_Bilbao
0,447.0,7096.0,43.0,73.0,49.0,196.0,65.41,10156.0,3813.0,6378.0,...,269.657312,269.657312,1036.0,97,0.0,226,0.0,0.0,0.0,0
1,449.0,7096.0,43.0,71.0,50.0,195.0,64.92,10437.0,3587.0,5890.0,...,269.763500,269.763500,1035.0,97,0.0,229,0.0,0.0,0.0,0
2,448.0,7099.0,43.0,73.0,50.0,196.0,64.48,9918.0,3508.0,5461.0,...,269.251688,269.251688,1036.0,97,1.0,224,0.0,0.0,0.0,0
3,438.0,7098.0,43.0,75.0,50.0,191.0,59.32,8859.0,3231.0,5238.0,...,269.203344,269.203344,1035.0,97,1.0,225,0.0,0.0,0.0,0
4,428.0,7097.0,43.0,74.0,42.0,189.0,56.04,8313.0,3499.0,4935.0,...,269.485500,269.485500,1035.0,97,1.0,221,0.0,0.0,0.0,0


In [7]:
nn = MinMaxScaler().fit_transform(df_global)
dataset = pd.DataFrame(nn, columns=df_global.columns)

In [8]:
inputs = [col for col in df_global.columns if col.startswith('x_')]
outputs = [col for col in df_global.columns if col.startswith('y_')]

x = dataset[inputs]
y = dataset[outputs]

In [9]:
# Define helper functions.
detector = IsolationForest(n_estimators=1000, random_state=SEED)
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

In [10]:
# Define some hyperparameters.
n_epochs, n_batches, n_samples = 20, 16, dataset.shape[0]
buffer_size, batch_size = n_samples, np.floor(n_samples/n_batches)

In [11]:
# Define training and test data sizes.
n_train, n_test = int(0.70*n_samples), int(0.30*n_samples)

In [12]:
# Define dataset instance.
data = tf.data.Dataset.from_tensor_slices((x.values, y.values))
data = data.shuffle(n_samples, reshuffle_each_iteration=True, seed=SEED)

In [13]:
# Define train and test data instances.
data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1)

In [14]:
# Define prior for regularization.

gauss_prior = tfd.Normal(
    loc = tf.zeros( len(outputs), dtype=tf.float64 ),
    scale = 1.0, #tf.ones( len(outputs), dtype=tf.float64 ),
    name = "p_normal",
    )

prior = tfd.Independent(
    gauss_prior,
    reinterpreted_batch_ndims = 1,
    name = "prior",
    )

regularizer = tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches, ) # Kullback–Leibler divergence (also called relative entropy)

In [15]:
def params_size(n) : return sum(range(n+2))-1 # N means + N(N+1)/2 co-variances

In [16]:
# Define model instance.
model = tfk.Sequential([

    tfk.layers.InputLayer(
        input_shape=(len(inputs),),
        name="input"
        ),

    # dense for inputs
    tfk.layers.Dense(
        n_batches,
        activation="relu",
        name="dense_input"
        ),

    # # a new dense for input
    # tfp.layers.DenseFlipout(
    #     10,
    #     activation="relu",
    #     name="dense_1"
    # )

    # dense for weights
    tfk.layers.Dense(
        params_size(len(outputs)), # uncertainty in the parameters weights
        activation=None,
        name="distribution_weights"
        ),

    # (declaration of the) posterior probability distribution structure
    tfp.layers.MultivariateNormalTriL(
        len(outputs),
        # activity_regularizer acts as prior for the output layer
        activity_regularizer=regularizer, 
        name="output"),
        
    ], name="model")

In [17]:
# Compile model.
model.compile(optimizer="adam", loss=neg_log_likelihood)

In [18]:
model_history = dict()

In [19]:
class CustomCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        print(f" ... i'm gonna pickle the model: ")
        for l in ['dense_input', 'distribution_weights', 'output']:
            tmp = model.get_layer(l)
            try:
                ump = list()
                for i in range(1000):
                    try:
                        ump.append(tmp.get_input_at(i))
                    except:
                        break
                model_history[epoch,l,'input'] = ump
            except:
                pass
            try:
                model_history[epoch,l,'weight'] = tmp.weights.copy()
            except:
                pass
            try:
                ump = list()
                for i in range(1000):
                    try:
                        ump.append(tmp.get_output_at(i))
                    except:
                        break
                model_history[epoch,l,'output'] = ump
            except:
                pass
            print(f'{epoch}th {l} state pickled!')
        # keys = list(logs.keys())
        # print('-+'*20)
        # print(f"End epoch {epoch} of training; got log keys: {keys}")
        # print('-+'*20)

In [20]:
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=True, callbacks=[CustomCallback()])

Epoch 1/20
239/240 [============================>.] - ETA: 0s - loss: 1.8502-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
End epoch 0 of training; got log keys: ['loss', 'val_loss']
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
240/240 [==============================] - 96s 379ms/step - loss: 1.8460 - val_loss: -2.9443
Epoch 2/20
239/240 [============================>.] - ETA: 0s - loss: -3.5769-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
End epoch 1 of training; got log keys: ['loss', 'val_loss']
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
240/240 [==============================] - 90s 376ms/step - loss: -3.5771 - val_loss: -3.9101
Epoch 3/20
239/240 [============================>.] - ETA: 0s - loss: -4.0596-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
End epoch 2 of training; got log keys: ['loss', 'val_loss']
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
240/240 [==============================] - 82s 343ms/step - loss: -4.0595 - val_loss: -4.1262
Epoch 4/20
239/240 [============================>.] - ETA: 0s -

In [21]:
# Describe model.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (Dense)         (None, 16)                896       
                                                                 
 distribution_weights (Dense  (None, 65)               1105      
 )                                                               
                                                                 
 output (MultivariateNormalT  ((None, 10),             0         
 riL)                         (None, 10))                        
                                                                 
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________


In [60]:
import pickle
pickle.dump({k:model_history[k] for k in model_history if 'weight' in k}, open('model.input.weght.output.history.20.epoch.pkl', 'wb'))